<h3>Imports</h3>

In [46]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier
import pickle
import matplotlib.pyplot as plt             #visualisation
%matplotlib inline 

<h3>Loading the data into the data frame</h3>

In [47]:
df = pd.read_csv("../Data/test.csv")

<h3>Descovering Data</h3>

In [48]:
df.head(5)
pass_id = df.PassengerId

In [49]:
#remove PassengerId, Name not important 
df = df.drop(["PassengerId","Name"],axis=1)
df.head(5)

,Pclass,Sex,Age,Ticket,Cabin,Embarked
0,3,male,34.5,330911,NaN,Q
1,3,female,47.0,363272,NaN,S
2,2,male,62.0,240276,NaN,Q
3,3,male,27.0,315154,NaN,S
4,3,female,22.0,3101298,NaN,S


In [50]:
df.dtypes


Pclass        int64
Sex          object
Age         float64
Ticket       object
Cabin        object
Embarked     object
dtype: object

In [51]:
df.isna().sum()

Pclass        0
Sex           0
Age          86
Ticket        0
Cabin       327
Embarked      0
dtype: int64

<ol>
    <li>Age,Cabin,Embarked has null values</li>
    <li>Age is Quantitative and Cabin,Embarked Categorical</li>
</ol>


<h3>Dealing with Categorized Data</h3>

In [52]:
#sex,ticket,cabin,Embarked are categorical data
#cabin,Embarked have missing data

In [53]:
percent_Cabin_missing = (df.isna().sum().Cabin)/len(df.Cabin)
print(percent_Cabin_missing)
#more than 70% drop Col
df = df.drop(["Cabin"],axis=1)

0.7822966507177034


In [54]:
#all Tickets has 1 to 4 survivors not important
df = df.drop(["Ticket"],axis=1)

In [55]:
#embarked has 3 unquie valuess given in problem C = Cherbourg, Q = Queenstown, S = Southampton
#sex has 2 unquie values given in problem female male
Embarked_mode = df.Embarked.mode()#S
df['Embarked'+"_was_missing"] = df['Embarked'].isna()
df['Embarked'] = df['Embarked'].fillna('S')
df['Embarked'] = df['Embarked'].replace(['C'],1)
df['Embarked'] = df['Embarked'].replace(['Q'],2)
df['Embarked'] = df['Embarked'].replace(['S'],3)
df['Sex'] = df['Sex'].replace(['male'],0)
df['Sex'] = df['Sex'].replace(['female'],1)


<h3>Dealing with Missing Data</h3>

In [56]:
#Age (Quantitative Data) only left dealt with cabin by removing and Embarked got the mode and added new columns
# Make copy to avoid changing original data (when imputing)
X = df#Features
X_test_plus = X.copy()

# # Make new columns indicating what will be imputed
X_test_plus["Age" + '_was_missing'] = X_test_plus["Age"].isnull()

# # Imputation
my_imputer = SimpleImputer()
imputed_X_test_plus = pd.DataFrame(my_imputer.fit_transform(X_test_plus))

# # Imputation removed column names; put them back
imputed_X_test_plus.columns = X_test_plus.columns

<h3>Using Model To Predict</h3>

In [57]:
filename = f'model.sav'
model = pickle.load(open(filename, 'rb'))

In [58]:
predictions = model.predict(imputed_X_test_plus)

<h3>Output File</h3>

In [59]:
output = pd.DataFrame({'PassengerId': pass_id,'Survived': predictions})
output.to_csv('../Data/Submissions/submission.csv', index=False)